In [1]:
import os

In [2]:
pwd

'/Users/allumariababu/Documents/PROJECT/Chicken-Disease-Classification/research'

In [3]:
os.chdir("../")

In [4]:
pwd

'/Users/allumariababu/Documents/PROJECT/Chicken-Disease-Classification'

In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [10]:
from ChickenDiseaseClassification.constants import *
from ChickenDiseaseClassification.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config


In [11]:
import os
import urllib.request as request
import zipfile
from ChickenDiseaseClassification import logger
from ChickenDiseaseClassification.utils.common import get_size

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        
        

In [12]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-12-06 20:05:36,052: INFO: common: YAML file: config/config.yaml loaded successfully]
[2023-12-06 20:05:36,054: INFO: common: YAML file: params.yaml loaded successfully]
[2023-12-06 20:05:36,055: INFO: common: Created directory at: artifacts]
[2023-12-06 20:05:36,056: INFO: common: Created directory at: artifacts/data_ingestion]
[2023-12-06 20:05:37,761: INFO: 2611842491: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 11794992
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "5842c7cba723332a405fe0cfdedb5085cd0d089bb3b4cdf22a5fbcc67e7b3fa9"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 7C3C:EDB8:4F2CD6:51AE02:6570D40F
Accept-Ranges: bytes
Date: Wed, 06 Dec 2023 20:05:37 GMT
Via: 1.1 varnish
X-Served-By: cache-lcy-eglc8600064-LCY
X-Ca